In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 2000 observations

In [2]:
n_sim, nobs, expo = 100, 2000, 250
err_pnl, popsize = 3e5, 5000

In [3]:
# Setting the true model
f1, f2 = bs.loss_model("Weibull", ["k1", "β1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])

f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

%time PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

CPU times: user 39.1 s, sys: 23.2 ms, total: 39.1 s
Wall time: 39.1 s


array([-174.49454405, -127.73007712,  -54.84021799])

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [4]:
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
def fit_one(k, f, f_name,  prior, X):
    trace, log_marg, DIC, WAIC = bs.smc(X, f, popsize, prior, verbose = False)
    PnLs = np.array(f.PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
    cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
    return([k, f_name , nobs, log_marg, trace["γ"].mean(), parms_true[-1] ] + cap.tolist() + true_cap.tolist())
def fit_all(k):
    print("simulation #"+ str(k))
    col_names = ["sim", "model_name", "nobs", "log_marg", "γ_map", "γ_true", "cap005", "cap01", "cap05", "cap005_true", "cap01_true", "cap05_true" ]
    df_temp = pd.DataFrame([fit_one(k, fs[i], f_names[i], prior_spliced_model[i], Xs[k]) for i in range(len(fs))], columns = col_names)
    df_temp['posterior_probability'] = np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)) / \
    np.sum(np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)))  
    γ_BMA = np.dot(df_temp['posterior_probability'].values, df_temp['γ_map'])
    cap_bma = np.dot(df_temp['posterior_probability'].values, df_temp[["cap005", "cap01", "cap05"]].values)
    BMA_df = pd.DataFrame([[k, "BMA" , nobs, None, γ_BMA, parms_true[-1] ] + cap_bma.tolist() + true_cap.tolist()],
            columns = col_names)
    df_temp = pd.concat([df_temp, BMA_df])
    return(df_temp)
%time res = Parallel(n_jobs= 50)(delayed(fit_all)(k) for k in range(n_sim))
pd.concat(res).to_csv("../../Data/Simulations/simu_wellspec_"+str(nobs)+".csv", sep=',')

CPU times: user 9.46 s, sys: 710 ms, total: 10.2 s
Wall time: 56min 21s


In [5]:
pd.concat(res)

,sim,model_name,nobs,log_marg,γ_map,γ_true,cap005,cap01,cap05,cap005_true,cap01_true,cap05_true,posterior_probability
0,0,Weibull_Log-Logistic,2000,-1666.405855,3.678970,1.5,-582.750455,-390.508207,-143.892007,-174.494544,-127.730077,-54.840218,3.085026e-01
1,0,Weibull_Lomax,2000,-1666.066528,3.023611,1.5,-479.880805,-330.621621,-132.464395,-174.494544,-127.730077,-54.840218,4.331385e-01
2,0,Weibull_Inverse-Weibull,2000,-1666.583236,3.999554,1.5,-649.860751,-422.905031,-156.872590,-174.494544,-127.730077,-54.840218,2.583589e-01
3,0,Inverse-Weibull_Log-Logistic,2000,-1701.793625,0.511770,1.5,-1873.747170,-1118.854480,-342.140443,-174.494544,-127.730077,-54.840218,1.319914e-16
4,0,Inverse-Weibull_Lomax,2000,-1692.482166,0.157433,1.5,-865.396683,-576.374678,-221.764902,-174.494544,-127.730077,-54.840218,1.460364e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,99,Weibull_Inverse-Weibull,2000,-1714.091283,2.045171,1.5,-565.732159,-364.142797,-128.144783,-174.494544,-127.730077,-54.840218,5.497459e-02
3,99,Inverse-Weibull_Log-Logistic,2000,-1772.949718,0.674711,1.5,-605.558714,-400.671341,-148.721796,-174.494544,-127.730077,-54.840218,1.507540e-27
4,99,Inverse-Weibull_Lomax,2000,-1765.063747,0.164825,1.5,-364.243091,-261.457849,-109.725626,-174.494544,-127.730077,-54.840218,4.009610e-24
5,99,Inverse-Weibull_Inverse-Weibull,2000,-1776.534502,1.079680,1.5,-734.118968,-467.512581,-153.503210,-174.494544,-127.730077,-54.840218,4.182314e-29
